In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType, IntegerType, DoubleType
from pyspark.sql.functions import *
import utils

In [103]:
spark = SparkSession \
        .builder \
        .appName("Nulls and Outliers Detection 1") \
        .getOrCreate()


In [104]:
df_test = spark.createDataFrame([('923,122,344.12','187,988', 'Porto', 10),('555,666,','888,444', "Coimbra",20), ('555,666.10,','888$,444', "Coimbra",11), ('555,666,','888,444', "Coimbra",8)], ['s1','s2', 's3', 'myint'])

In [106]:
df_temp = df = spark.read.csv(path = 'GROUP7/bss9-579f_clean/part-00000-919430be-1ef9-4fee-9d5f-c8cac19198b3-c000.csv', header = True,inferSchema = True, sep='\t')

In [120]:
df_temp.count()

480

In [105]:
df_test.show()

+--------------+--------+-------+-----+
|            s1|      s2|     s3|myint|
+--------------+--------+-------+-----+
|923,122,344.12| 187,988|  Porto|   10|
|      555,666,| 888,444|Coimbra|   20|
|   555,666.10,|888$,444|Coimbra|   11|
|      555,666,| 888,444|Coimbra|    8|
+--------------+--------+-------+-----+



In [56]:
replaced_df = df_test
for col, dtype in replaced_df.dtypes:
    print(f"### {col} ###")
    if 'string' in dtype:
        print(f"\tIs string!")
        tmp_col = f"{col}_temp"
        print(f"\t{replaced_df.select(col).limit(5).rdd.map(lambda x: x[0]).collect()}")
        replaced_df = replaced_df.withColumn(tmp_col, utils.replace_commas_symbols_udf(replaced_df[col]).cast(DoubleType()))
        if utils.is_double(replaced_df, tmp_col):
            print(f"\tIs double actually!")
            replaced_df = replaced_df.withColumn(col, replaced_df[tmp_col])
        replaced_df = replaced_df.drop(tmp_col)
        print(f"\t{replaced_df.select(col).limit(5).rdd.map(lambda x: x[0]).collect()}")

### s1 ###
	Is string!
	['923,122,344.12', '555,666,', '555,666.10,', '555,666,']
	Is double actually!
	[923122344.12, 555666.0, 555666.1, 555666.0]
### s2 ###
	Is string!
	['187,988', '888,444', '888$,444', '888,444']
	Is double actually!
	[187988.0, 888444.0, None, 888444.0]
### s3 ###
	Is string!
	['Porto', 'Coimbra', 'Coimbra', 'Coimbra']
	['Porto', 'Coimbra', 'Coimbra', 'Coimbra']
### myint ###


In [16]:
df_clean = replaced_df
null_val = 'NONE_VALUE_'
print(f"Before: {df_test.count()}")
for col, dtype in df_clean.dtypes:
    if 'string' not in dtype:
        print(f"{col} not string: {dtype}")
        print("--"*10)
        continue
    df_clean = df_clean.fillna(null_val, col)
    if df_clean.select(col).distinct().count() > 5:
        print(f"#### {col} (>5) ####")
        print(f"Before: {df_clean.count()}")
        df_clean = df_clean.filter(f"{col} != 'NONE_VALUE_'")
        print(f"After {col}: {df_clean.count()}\n")
    else:
        print(f"#### {col} (<5) ####")
    l = df_clean.select(col).distinct().rdd.map(lambda x: x[0])
    print(f"distinct count: {l.count()}\n")
    for v in sorted(l.collect()):
        print(v)
    print('\n'*4)
    print("--"*10)

Before: 4
s1 not string: double
--------------------
s2 not string: double
--------------------
#### s3 (<5) ####
distinct count: 2

Coimbra
Porto





--------------------
myint not string: bigint
--------------------


In [93]:
df_clean2 = replaced_df
string_cols = [col for col, dtype in df_clean2.dtypes if 'string' in dtype]
df_clean2 = df_clean2.dropna(subset=string_cols)

In [94]:
med = df_clean2.approxQuantile("s1", [0.25,0.75], 0)

In [95]:
cols = ['s1','s2','myInt']
bounds = {}
for col in cols:
    quantiles = df_clean2.approxQuantile(col,[0.25,0.75],0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5 * IQR,quantiles[1] + 1.5 * IQR]

In [96]:
bounds

{'myInt': [3.5, 15.5],
 's1': [555665.8500000001, 555666.25],
 's2': [-862696.0, 1939128.0]}

In [97]:
df_clean2 = df_clean2.withColumn("id", monotonically_increasing_id())


In [98]:
df_clean2.show()

+--------------+--------+-------+-----+-----------+
|            s1|      s2|     s3|myint|         id|
+--------------+--------+-------+-----+-----------+
|9.2312234412E8|187988.0|  Porto|   10|          0|
|      555666.0|888444.0|Coimbra|   20| 8589934592|
|      555666.1|    null|Coimbra|   11|17179869184|
|      555666.0|888444.0|Coimbra|    8|25769803776|
+--------------+--------+-------+-----+-----------+



In [100]:
outliers = df_clean2.select(*['id'] + [((df_clean2['s1'] < bounds['s1'][0]) | (df_clean2['s1'] > bounds['s1'][1])).alias('s1'+'_o')])
outliers.show()

+-----------+-----+
|         id| s1_o|
+-----------+-----+
|          0| true|
| 8589934592|false|
|17179869184|false|
|25769803776|false|
+-----------+-----+



In [71]:
df_outliers = df_clean2.join(outliers, on='id')
df_outliers.filter('s1_o').select('id', 's1').show()
df_outliers.filter('s2_o').select('id', 's2').show()
df_outliers.filter('myInt_o').select('id', 'myInt').show()

+---+--------------+
| id|            s1|
+---+--------------+
|  0|9.2312234412E8|
+---+--------------+

+---+---+
| id| s2|
+---+---+
+---+---+

+----------+-----+
|        id|myInt|
+----------+-----+
|8589934592|   20|
+----------+-----+



In [75]:
df = df_clean2
cols_to_check = []
bounds = {}
for col,dtype in df.dtypes:
    if 'string' not in dtype and col!='id':
        cols_to_check.append(col)
        quantiles = df.approxQuantile(col,[0.25,0.75],0.05)
        IQR = quantiles[1] - quantiles[0]
        bounds[col] = [quantiles[0] - 1.5 * IQR,quantiles[1] + 1.5 * IQR]
        
df = df.withColumn("id", monotonically_increasing_id())
outliers = df.select(*['id'] + [((df[c] < bounds[c][0]) | (df[c] > bounds[c][1])).alias(c+'_o') for c in cols_to_check])
outliers.show()

df = df.join(outliers, on='id')
#print(cols_to_check)
for col in cols_to_check:
    df_outliers.filter(col + '_o').select('id', col).show()
    





+-----------+-----+-----+-------+
|         id| s1_o| s2_o|myint_o|
+-----------+-----+-----+-------+
|          0| true|false|  false|
| 8589934592|false|false|   true|
|17179869184|false| null|  false|
|25769803776|false|false|  false|
+-----------+-----+-----+-------+

['s1', 's2', 'myint']
+---+--------------+
| id|            s1|
+---+--------------+
|  0|9.2312234412E8|
+---+--------------+

+---+---+
| id| s2|
+---+---+
+---+---+

+----------+-----+
|        id|myint|
+----------+-----+
|8589934592|   20|
+----------+-----+



In [77]:
'id' in df.columns

True

In [126]:
def get_outliers(df,col):
    bounds = {}
    quantiles = df.approxQuantile(col,[0.25,0.75],0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 2 * IQR,quantiles[1] + 2 * IQR]
    #print(bounds)
    if 'rid'not in df.columns:
        df = df.withColumn("rid", monotonically_increasing_id())
    outliers = df.select(*['rid'] + [((df[col] < bounds[col][0]) | (df[col] > bounds[col][1])).alias(col+'_o')])
    df_outliers = df.join(outliers, on='rid')
    df_outliers = df_outliers.filter(col + '_o').select('rid', col)
    return df_outliers


In [127]:
df_count = df_temp.count() 

In [130]:
keyval = {}
for col,dtype in df_temp.dtypes:
    if 'string' not in dtype and col!='rid':
        
        keyval[col] = get_outliers(df_temp,col)
        print(col , " counts:",keyval[col].count(),' % counts = {}%'.format(100*(keyval[col].count()/df_count)))
        #print()
        
        

comparable_rental_2_gross_sqft  counts: 45  % counts = 9.375%
brooklyn_condominiums_comparable_properties_estimated_expense  counts: 46  % counts = 9.583333333333334%
brooklyn_condominiums_comparable_properties_total_units  counts: 48  % counts = 10.0%
brooklyn_condominiums_comparable_properties_expense_per_sqft  counts: 5  % counts = 1.0416666666666665%
comparable_rental_2_gross_income_per_sqft  counts: 9  % counts = 1.875%
comparable_rental_1_gross_income_per_sqft  counts: 4  % counts = 0.8333333333333334%
comparable_rental_1_estimated_expense  counts: 53  % counts = 11.041666666666666%
comparable_rental_1_net_operating_income  counts: 63  % counts = 13.125%
comparable_rental_2_estimated_expense  counts: 52  % counts = 10.833333333333334%
brooklyn_condominiums_comparable_properties_gross_sqft  counts: 45  % counts = 9.375%
brooklyn_condominiums_comparable_properties_market_value_per_sqft  counts: 3  % counts = 0.625%
brooklyn_condominiums_comparable_properties_gross_income_per_sqft  

In [ ]:
clusters = KM